In [1]:
# install the latest released version of PennyLane
!pip install pennylane --upgrade

  Created wheel for autograd: filename=autograd-1.3-py3-none-any.whl size=47989 sha256=8f928093c3e40a807184c806aeccf673c28d2afd6dbc6b2150ad626506bf95a0
  Stored in directory: c:\users\0\appdata\local\pip\cache\wheels\85\f5\d2\3ef47d3a836b17620bf41647222825b065245862d12aa62885
Successfully built autograd


In [2]:
%matplotlib inline

# Pennylane

## Quantum circuits

In PennyLane, quantum computations are represented as *quantum node* objects. A quantum node is used to declare the quantum circuit, and also ties the computation to a specific device that executes it. Quantum nodes can be easily created by using the `qnode` decorator.

QNodes can interface with any of the supported numerical and machine learning libraries — `NumPy`, `PyTorch`, `TensorFlow`, and `JAX` — indicated by providing an optional `interface` argument when creating a QNode. Each interface allows the quantum circuit to integrate seamlessly with library-specific data structures (e.g., NumPy arrays, or Pytorch/TensorFlow tensors) and `optimizers`.

By default, QNodes use the NumPy interface. 



### Quantum functions

A quantum circuit is constructed as a special Python function, a *quantum circuit function*, or *quantum function* in short. For example:

In [3]:
import pennylane as qml
import numpy as np

def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(y, wires=1)
    return qml.expval(qml.PauliZ(1))

Note

PennyLane uses the term `wires` to refer to a quantum subsystem — for most devices, this corresponds to a qubit. For continuous-variable devices, a wire corresponds to a quantum mode.

Quantum functions are a restricted subset of Python functions, adhering to the following constraints:

- The quantum function accepts classical inputs, and consists of `quantum operations` or sequences of operations called `Templates`, using one instruction per line.

- The function can contain classical flow control structures such as `for` loops, but in general they must not depend on the parameters of the function.

- The quantum function must always return either a single or a tuple of *measured observable values*, by applying a `measurement function` to a `qubit observable` or `continuous-value observable`.

Note

Measured observables **must** come after all other operations at the end of the circuit function as part of the return statement, and cannot appear in the middle.

Note

Quantum functions can only be evaluated on a device from within a QNode.




### Defining a device

To run — and later optimize — a quantum circuit, one needs to first specify a *computational device*.

The device is an instance of the `Device` class, and can represent either a simulator or hardware device. They can be instantiated using the `device` loader.

In [4]:
dev = qml.device('default.qubit', wires=2, shots=1000)

PennyLane offers some basic devices such as the `default.qubit` and `default.gaussian` simulators. Note that the choice of a device significantly determines the speed of your computation, as well as the available options that can be passed to the device loader.



#### Device options

When loading a device, the name of the device must always be specified. Further options can then be passed as keyword arguments; these options can differ based on the device. For the in-built `default.qubit` and `default.gaussian` devices, the options are:

- `wires` (*int* or *Iterable*): Number of subsystems represented by the device, or iterable that contains unique labels for the subsystems as numbers (i.e., [-1, 0, 2]) and/or strings (['ancilla', 'q1', 'q2']).

- `shots` (*None* or *int* or *list[int]*): How many times the circuit should be evaluated (or sampled) to estimate statistical quantities. On some supported simulator devices, `shots=None` indicates to compute measurement statistics *exactly*. Note that this argument can be temporarily overwritten when a QNode is called. For example, `my_qnode(shots=3)` will temporarily evaluate `my_qnode` using three shots.



#### Shot batches

Batches of shots can be specified by passing a list, allowing measurement statistics to be course-grained with a single QNode evaluation.

Consider

In [14]:
shots_list = [5, 10, 1000]
dev = qml.device("default.qubit", wires=2, shots=shots_list)

When QNodes are executed on this device, a single execution of 1015 shots will be submitted. However, three sets of measurement statistics will be returned; using the first 5 shots, second set of 10 shots, and final 1000 shots, separately.

For example:


In [15]:
@qml.qnode(dev)
def circuit(x):
  qml.RX(x, wires=0)
  qml.CNOT(wires=[0, 1])
  return qml.expval(qml.PauliZ(0) @ qml.PauliX(1)), qml.expval(qml.PauliZ(0))

Executing this, we will get an output of size `(3, 2)`:

In [16]:
circuit(0.5)

tensor([[-0.2  ,  1.   ],
        [-0.4  ,  1.   ],
        [ 0.022,  0.906]], requires_grad=True)

#### Custom wire labels

When you create a device by passing an integer to the `wires` argument, the integer defines the *number* of *wires* that you can address by consecutive integer labels `0, 1, 2, ....`

But you can define your own wire labels instead, which may be handy if wires have “meanings” like an ancilla or garbage register, if they are arranged in a non-linear fashion like a grid, or if there are wires that you want to skip because they do not work on a hardware device.

This is done by passing an iterable of wire labels to the `wires` argument:

In [17]:
dev = qml.device('default.qubit', wires=['wire1', 'wire2'], shots=1000)

In the quantum function you can now use your own labels to address wires:

In [18]:
def my_quantum_function(x, y):
    qml.RZ(x, wires='wire1')
    qml.CNOT(wires=['wire1' ,'wire2'])
    qml.RY(y, wires='wire2')
    return qml.expval(qml.PauliZ('wire2'))

Allowed wire labels can be of the following types:

- *strings* like `wires=['a', 'd', 'b', ...]` or `wires=['ancilla', 'q1', 'q2', ...]`

- *integers* like `wires=[0, 4, 7]` or even `wires=[-1, 0, 4]`

- *floats* and other *numbers* like `wires=[1., 2., 4.]`

- *mixed* types like `wires=['ancilla', -1, 0, 'q3']`

Note

Some devices, such as hardware chips, may have a fixed number of wires. The iterable of labels passed to the device’s `wires` argument must match this expected number of wires.



### Creating a quantum node

Together, a quantum function and a device are used to create a *quantum node* or `QNode` object, which wraps the quantum function and binds it to the device.

A QNode can be explicitly created as follows:

In [19]:
circuit = qml.QNode(my_quantum_function, dev)

The QNode can be used to compute the result of a quantum circuit as if it was a standard Python function. It takes the same arguments as the original quantum function:

In [20]:
circuit(np.pi/4, 0.7)

tensor(0.774, requires_grad=True)

To view the quantum circuit after it has been executed, we can use the `draw()` method:

In [21]:
print(circuit.draw())

 wire1: ──RZ(0.785)──╭C───────────┤     
 wire2: ─────────────╰X──RY(0.7)──┤ ⟨Z⟩ 



### The QNode decorator

A more convenient — and in fact the recommended — way for creating QNodes is the provided `qnode` decorator. This decorator converts a Python function containing PennyLane quantum operations to a `QNode` circuit that will run on a quantum device.

Note

The decorator completely replaces the Python-based quantum function with a `QNode` of the same name — as such, the original function is no longer accessible.

For example:

In [22]:
dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def circuit(x):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(x, wires=1)
    return qml.expval(qml.PauliZ(1))

result = circuit(0.543)

In [23]:
result

tensor(0.85616242, requires_grad=True)

### Collections of QNodes

Sometimes you may need multiple QNodes that only differ in the measurement observable (like in VQE), or in the device they are run on (for example, if you benchmark different devices), or even the quantum circuit that is evaluated. While these QNodes can be defined manually “by hand”, PennyLane offers **QNode collections** as a convenient way to define and run families of QNodes.

QNode collections are a sequence of QNodes that:

1. Have the same function signature, and

2. Can be evaluated independently (that is, the input of any QNode in the collection does not depend on the output of another).

Consider the following two quantum nodes:

In [24]:
dev1 = qml.device('default.qubit', wires=2)

@qml.qnode(dev1)
def x_rotations(params):
    qml.RX(params[0], wires=0)
    qml.RX(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

dev2 = qml.device('default.qubit', wires=2)

@qml.qnode(dev2)
def y_rotations(params):
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.Hadamard(0))

As the QNodes in the collection have the same signature, and we can can construct a `QNodeCollection` and therefore feed them the same parameters:

In [25]:
qnodes = qml.QNodeCollection([x_rotations, y_rotations])
len(qnodes)

2

In [26]:
qnodes([0.2, 0.1])

array([0.98006658, 0.70703636])

PennyLane also provides some high-level tools for creating and evaluating QNode collections. For example, `map()` allows a single function of quantum operations (or `template`) to be mapped across multiple observables or devices.

For example, consider the following quantum function ansatz:

In [27]:
def my_ansatz(params, **kwargs):
    qml.RX(params[0], wires=0)
    qml.RX(params[1], wires=1)
    qml.CNOT(wires=[0, 1])

We can define a list of observables, and two devices:

In [28]:
obs_list = [qml.PauliX(0) @ qml.PauliZ(1), qml.PauliZ(0) @ qml.PauliX(1)]
dev1 = qml.device('default.qubit', wires=2)
dev2 = qml.device('default.qubit', wires=2)

Mapping the template across the observables and devices creates a `QNodeCollection`:

In [29]:
qnodes = qml.map(my_ansatz, obs_list, [dev1, dev2], measure="expval")
type(qnodes)

pennylane.collections.qnode_collection.QNodeCollection

In [30]:
params = [0.54, 0.12]
qnodes(params)

array([-0.06154835,  0.        ])

Functions are available to process QNode collections, including `dot()`, `sum()`, and `apply()`:

In [41]:
cost_fn = qml.sum(qnodes)
cost_fn(params)

-0.061548354407471695

Note

QNode collections support an experimental parallel execution mode. See the `QNodeCollection` documentation for more details.



## Quantum operations

PennyLane supports a wide variety of quantum operations—such as gates, noisy channels, state preparations and measurements. These operations can be used exclusively in quantum functions, like shown in the following example:

In [42]:
import pennylane as qml

def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(y, wires=1)
    qml.T(wires=0).inv()
    qml.AmplitudeDamping(0.1, wires=0)
    return qml.expval(qml.PauliZ(1))

This quantum function uses the `RZ`, `CNOT`, `RY` gates, the `AmplitudeDamping` noisy channel as well as the `PauliZ` observable.

Note that PennyLane supports inverting quantum operations via the `Op(param, wires).inv()` method. Additionally, PennyLane provides a function `qml.inv` that can be used to invert sequences of operations and Templates.

A list of all quantum operations supported by PennyLane can be foung at https://pennylane.readthedocs.io/en/stable/introduction/operations.html.



## Measurements

PennyLane can extract different types of measurement results from quantum devices: the expectation of an observable, its variance, samples of a single measurement, or computational basis state probabilities.

For example, the following circuit returns the expectation value of the `PauliZ` observable on wire 1:

In [43]:
def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(y, wires=1)
    return qml.expval(qml.PauliZ(1))

The available measurement functions are

- `expval`(op) - Expectation value of the supplied observable.

- `sample`(op) - Sample from the supplied observable, with the number of shots determined from the `dev.shots` attribute of the corresponding device.

- `var`(op) - Variance of the supplied observable.

- `probs`(wires) - Probability of each computational basis state.

- `state`() - Quantum state in the computational basis.

- `density_matrix`(wires) - Quantum density matrix in the computational basis.

Note

All measurement functions support analytic differentiation, with the exception of `sample()`, as it returns stochastic results.



### Combined measurements

Quantum functions can also return combined measurements of multiple observables, as long as each wire is not measured more than once:

In [44]:
def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(y, wires=1)
    return qml.expval(qml.PauliZ(1)), qml.var(qml.PauliX(0))

You can also use list comprehensions, and other common Python patterns:

In [45]:
def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(y, wires=1)
    return [qml.expval(qml.PauliZ(i)) for i in range(2)]

As a full example of combined measurements, let us look at a Bell state $\frac{|00\rangle + |11\rangle}{\sqrt{2}}$
, prepared by a `Hadamard` and `CNOT` gate.

In [50]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=2, shots=1000)

@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

The combined PauliZ-measurement of the first and second qubit returns a list of two lists, each containing the measurement results of the respective qubit. As a default, `sample`() returns 1000 samples per observable.

In [51]:
result = circuit()
result.shape

(2, 1000)

In [52]:
print(circuit.draw())

 0: ──H──╭C──┤ Sample[Z] 
 1: ─────╰X──┤ Sample[Z] 



Since the two qubits are maximally entangled, the measurement results always coincide, and the lists are therefore equal:

In [53]:
np.all(result[0] == result[1])

True

### Tensor observables

PennyLane supports measuring the tensor product of observables, by using the `@` notation. For example, to measure the expectation value of $Z ⊗ I ⊗ X$:

In [55]:
def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(y, wires=1)
    qml.CNOT(wires=[0, 2])
    return qml.expval(qml.PauliZ(0) @ qml.PauliX(2))

Note that we don’t need to declare the identity observable on wire 1; this is implicitly assumed.

The tensor observable notation can be used inside all measurement functions that accept observables as arguments, including `expval`(), `var`(), and `sample`().



### Probability

You can also train QNodes on computational basis probabilities, by using the `probs`() measurement function. Unlike other measurement functions, **this does not accept observables**. Instead, it will return a flat array or tensor containing the (marginal) probabilities of each quantum state.

In [56]:
def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(y, wires=1)
    qml.CNOT(wires=[0, 2])
    return qml.probs(wires=[0, 1])

For example:

In [57]:
dev = qml.device("default.qubit", wires=3)
qnode = qml.QNode(my_quantum_function, dev)
qnode(0.56, 0.1)

tensor([0.99750208, 0.00249792, 0.        , 0.        ], requires_grad=True)

The returned probability array uses lexicographical ordering, so corresponds to a 99.75% probability of measuring state $|00\rangle$, and a 0.25% probability of measuring state $|01\rangle$.



### Changing the number of shot

For hardware devices where the number of shots determines the accuracy of the expectation value and variance, as well as the number of samples returned, it can sometimes be convenient to execute the same QNode with differing number of shots.

For simulators like `default.qubit`, finite shots will be simulated if we set `shots` to a positive integer.

The shot number can be changed on the device itself, or temporarily altered by the `shots` keyword argument when executing the QNode:

In [58]:
dev = qml.device("default.qubit", wires=1, shots=10)

@qml.qnode(dev)
def circuit(x, y):
    qml.RX(x, wires=0)
    qml.RY(y, wires=0)
    return qml.expval(qml.PauliZ(0))

# execute the QNode using 10 shots
result = circuit(0.54, 0.1)

# execute the QNode again, now using 1 shot
result = circuit(0.54, 0.1, shots=1)

With an increasing number of shots, the average over measurement samples converges to the exact expectation of an observable. Consider the following circuit:

In [59]:
# fix seed to make results reproducable
np.random.seed(1)

dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    return qml.expval(qml.PauliZ(0))

Running the simulator with `shots=None` returns the exact expectation.

In [60]:
circuit(shots=None)

tensor(0., requires_grad=True)

Now we set the device to return stochastic results, and increase the number of shots starting from 10.

In [61]:
circuit(shots=10)

tensor(0.6, requires_grad=True)

In [62]:
circuit(shots=1000)

tensor(-0.016, requires_grad=True)

In [63]:
circuit(shots=100000)

tensor(0.00046, requires_grad=True)

The result converges to the exact expectation.

## Templates

PennyLane provides a growing library of pre-coded templates of common variational circuit architectures that can be used to easily build, evaluate, and train more complex models. In the literature, such architectures are commonly known as an *ansatz*.

PennyLane conceptually distinguishes different types of templates, such as `Embeddings`, `Layers`, `State preparations` and `Subroutines`.

Most templates are complemented by functions that provide an array of random `initial parameters`.

Full gallery of built-in templates provided by PennyLane can be found at https://pennylane.readthedocs.io/en/stable/introduction/templates.html

